In [ ]:
from transformers import AutoFeatureExtractor, AutoTokenizer
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
image_processor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-384")
decoder_tokenizer = AutoTokenizer.from_pretrained("urduhack/roberta-urdu-small")

#processor = TrOCRProcessor(feature_extractor=feature_extractor, tokenizer=decoder_tokenizer)
processor = TrOCRProcessor(image_processor=image_processor, tokenizer=decoder_tokenizer)

In [ ]:
# processor.save_pretrained('../test/test_processor')
processor.save_pretrained('./preprocessor')

In [ ]:
import pandas as pd
import cv2
df = pd.read_excel("./data/sen_dataset.xlsx", header=None)
df.rename(columns={0: "filename", 1: "text"}, inplace=True)
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.3)

train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [ ]:
test_df.head()

In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image

class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text 
        file_name = self.df['filename'][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(self.root_dir + file_name).convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, padding="max_length",max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

In [ ]:
from transformers import TrOCRProcessor

processor = TrOCRProcessor.from_pretrained("./preprocessor/")

train_dataset = IAMDataset(root_dir= "./data/images/",
                           df = train_df,
                           processor=processor)

eval_dataset = IAMDataset(root_dir= "./data/images/",
                           df = test_df,
                           processor = processor)

In [ ]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))

In [ ]:
image = Image.open(train_dataset.root_dir + train_df['filename'][6]).convert("RGB")
image

In [ ]:
encoding = train_dataset[6]
for k,v in encoding.items():
    print(k, v.shape)
    
labels = encoding['labels']
labels[labels == -100] = processor.tokenizer.pad_token_id
label_str = processor.decode(labels, skip_special_tokens=True)
print(label_str)

In [ ]:
from transformers import VisionEncoderDecoderModel

model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained("google/vit-base-patch16-384", 
                                                                  "urduhack/roberta-urdu-small")

# set decoder config to causal lm
model.config.decoder.is_decoder = True
model.config.decoder.add_cross_attention = True

# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id

# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

# Evaluating using epoch

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
   evaluation_strategy = "epoch",
   learning_rate= 0.002,
   per_device_train_batch_size = 32,
   per_device_eval_batch_size = 8,
   weight_decay = 0.01,
   save_total_limit = 2,
   num_train_epochs = 40,
   output_dir = "./train/",
   predict_with_generate=True,  
)

In [ ]:
import evaluate
cer_metric = evaluate.load('cer')

In [ ]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [ ]:
from transformers import default_data_collator

# instantiate trainer
trainer = Seq2SeqTrainer(
                        model = model,
                        tokenizer = processor.feature_extractor,
                        args = training_args,
                        compute_metrics = compute_metrics,
                        train_dataset = train_dataset,
                        eval_dataset = eval_dataset,
                        data_collator = default_data_collator
                        )
trainer.train()

In [ ]:
trainer.save_model('./trainings/models')

In [ ]:
model = VisionEncoderDecoderModel.from_pretrained("./trainer")

In [ ]:
image = Image.open('./dat').convert("RGB")

In [ ]:
image

In [ ]:
pixel_values = processor.feature_extractor(image, return_tensors="pt").pixel_values 
print(pixel_values.shape)

In [ ]:
generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)

In [ ]:
test = Image.open('/content/8.png').convert("RGB")
testvalues = processor.feature_extractor(test, return_tensors="pt").pixel_values 
print(testvalues.shape)

In [ ]:
testids = model.generate(testvalues)
testtext = processor.batch_decode(testids, skip_special_tokens=True)[0]
print(testtext)